In [136]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [153]:
df = pd.read_csv('blockchain_dot_com_daily_data.csv')

In [162]:
data = df[['datetime', 'market_price_usd', 'hash_rate']]
data['diff_usd'] = (data['market_price_usd'] / data['market_price_usd'].shift(1))
data['log_usd'] = np.log(data['diff_usd'])
data['diff_hash'] = (data['hash_rate'] / data['hash_rate'].shift(1))
data['log_hash'] = np.log(data['diff_hash'])
data

/var/folders/_5/jcypzwvd5fz1trv2_rv7m_7r0000gn/T/ipykernel_33002/2437917970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['diff_usd'] = (data['market_price_usd'] / data['market_price_usd'].shift(1))
/var/folders/_5/jcypzwvd5fz1trv2_rv7m_7r0000gn/T/ipykernel_33002/2437917970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['log_usd'] = np.log(data['diff_usd'])
/var/folders/_5/jcypzwvd5fz1trv2_rv7m_7r0000gn/T/ipykernel_33002/2437917970.py:4: SettingWithCopyWarning: 
A value is trying to be 

,datetime,market_price_usd,hash_rate,diff_usd,log_usd,diff_hash,log_hash
0,2009-01-03,0.00,4.971027e-08,NaN,NaN,NaN,NaN
1,2009-01-04,0.00,0.000000e+00,NaN,NaN,0.000000,-inf
2,2009-01-05,0.00,0.000000e+00,NaN,NaN,NaN,NaN
3,2009-01-06,0.00,0.000000e+00,NaN,NaN,NaN,NaN
4,2009-01-07,0.00,0.000000e+00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5351,2023-08-29,26108.31,4.368632e+08,1.000749,0.000748,1.338983,0.291910
5352,2023-08-30,27731.23,3.483846e+08,1.062161,0.060306,0.797468,-0.226313
5353,2023-08-31,27301.12,3.981538e+08,0.984490,-0.015631,1.142857,0.133531
5354,2023-09-01,25932.99,4.175085e+08,0.949887,-0.051412,1.048611,0.047467


In [163]:
data_log = data.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()
data_log

,datetime,market_price_usd,hash_rate,diff_usd,log_usd,diff_hash,log_hash
593,2010-08-19,0.07,3.561656e-03,1.000000,0.000000,1.052632,0.051293
594,2010-08-20,0.07,4.248547e-03,1.000000,0.000000,1.192857,0.176351
595,2010-08-21,0.07,4.681034e-03,1.000000,0.000000,1.101796,0.096942
596,2010-08-22,0.07,5.062640e-03,1.000000,0.000000,1.081522,0.078369
597,2010-08-23,0.07,5.520567e-03,1.000000,0.000000,1.090452,0.086593
...,...,...,...,...,...,...,...
5351,2023-08-29,26108.31,4.368632e+08,1.000749,0.000748,1.338983,0.291910
5352,2023-08-30,27731.23,3.483846e+08,1.062161,0.060306,0.797468,-0.226313
5353,2023-08-31,27301.12,3.981538e+08,0.984490,-0.015631,1.142857,0.133531
5354,2023-09-01,25932.99,4.175085e+08,0.949887,-0.051412,1.048611,0.047467


In [164]:
df

,datetime,mempool_size,transaction_rate,market_cap_usd,average_block_size,market_price_usd,exchange_volume_usd,average_confirmation_time,hash_rate,difficulty,miners_revenue,total_transaction_fees
0,2009-01-03,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,4.971027e-08,1.000000e+00,0.000000e+00,0.000000
1,2009-01-04,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2,2009-01-05,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
3,2009-01-06,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
4,2009-01-07,0.00,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5351,2023-08-29,92829238.00,4.366667,5.085026e+11,1.658419,26108.31,7.316306e+07,58.918310,4.368632e+08,5.562144e+13,2.689015e+07,21.151151
5352,2023-08-30,24732393.50,3.658333,5.339097e+11,1.623768,27731.23,2.914660e+08,29.863134,3.483846e+08,5.562144e+13,2.209441e+07,19.220988
5353,2023-08-31,51403039.75,3.766667,5.300776e+11,1.648731,27301.12,1.246572e+08,40.942787,3.981538e+08,5.562144e+13,2.496186e+07,26.540640
5354,2023-09-01,73927621.00,3.808333,5.064906e+11,1.688033,25932.99,1.474262e+08,58.477930,4.175085e+08,5.562144e+13,2.512079e+07,26.151918


In [157]:
X = df['hash_rate'].values.reshape(-1, 1)
y = df['market_price_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
results = model.fit(X_train, y_train)
y_pred = lr.predict(X_test)

results.coef_, results.score(X, y)

(array([0.00011382]), 0.5528226165117915)

In [158]:
mse_lr = mean_squared_error(y_test, y_pred)

print("MSE линейной регрессии:", mse_lr)
sgd = SGDRegressor()
sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

mse_sgd = mean_squared_error(y_test, y_pred_sgd)
print("MSE SGDRegressor:", mse_sgd)

mse_lr > mse_sgd

MSE линейной регрессии: 88508492.52103077
MSE SGDRegressor: 7.675277615303234e+49


False

In [159]:
result = pd.DataFrame([y_test, lr.predict(X_test)]).T

In [160]:
result

,market_price_usd,Unnamed 0
5010,18507.36,NaN
1650,87.90,NaN
401,0.00,23468.977582
4222,9707.50,NaN
1684,109.35,NaN
...,...,...
4746,46249.56,NaN
1592,117.99,2478.193096
4932,20181.91,NaN
4015,7168.31,NaN


In [161]:
result = pd.DataFrame([y_test, sgd.predict(X_test)]).T
result

,market_price_usd,Unnamed 0
5010,18507.36,NaN
1650,87.90,NaN
401,0.00,-1.515366e+25
4222,9707.50,NaN
1684,109.35,NaN
...,...,...
4746,46249.56,NaN
1592,117.99,-2.144649e+21
4932,20181.91,NaN
4015,7168.31,NaN


In [167]:
X = data_log['log_hash'].values.reshape(-1, 1)
y = data_log['market_price_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
results = model.fit(X_train, y_train)
y_pred = lr.predict(X_test)

results.coef_, results.score(X, y)

(array([-587.14058433]), 0.000132720463568492)

In [169]:
mse_lr = mean_squared_error(y_test, y_pred)

print("MSE линейной регрессии:", mse_lr)
sgd = SGDRegressor()
sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)

mse_sgd = mean_squared_error(y_test, y_pred_sgd)
print("MSE SGDRegressor:", mse_sgd)

MSE линейной регрессии: 256414869.59175032
MSE SGDRegressor: 209040701.19600126


In [170]:
result = pd.DataFrame([y_test, lr.predict(X_test)]).T
result

,market_price_usd,Unnamed 0
1421,12.43,2475.221893
1654,94.69,NaN
1824,732.00,NaN
4158,9167.26,NaN
2010,631.00,NaN
...,...,...
5140,23755.85,NaN
892,23.66,2475.221900
1596,123.32,NaN
975,8.56,2475.221913


In [171]:
X = data_log['log_hash'].values.reshape(-1, 1)
y = data_log['log_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression()
results = model.fit(X_train, y_train)
y_pred = lr.predict(X_test)

results.coef_, results.score(X, y)

(array([0.00067238]), 1.8586044517321376e-05)

In [172]:
mse_lr = mean_squared_error(y_test, y_pred)

print("MSE линейной регрессии:", mse_lr)
sgd = SGDRegressor()
sgd.fit(X_train, y_train)

y_pred_sgd = sgd.predict(X_test)
mse_sgd = mean_squared_error(y_test, y_pred_sgd)
print("MSE SGDRegressor:", mse_sgd)

MSE линейной регрессии: 6126708.434743441
MSE SGDRegressor: 0.0027641935046196567
